In [4]:
import numpy as np
import pandas as pd

In [10]:
import os 
import sys

# count the number of images in the directory
image_dir = "../data/images/pixilart/"
image_files = os.listdir(image_dir)
num_images = len(image_files)
print("Number of images: ", num_images)


Number of images:  41716


In [18]:
import csv

csv_file = "../data/pixilart.csv"

with open(csv_file, 'r') as lines:
    for l in csv.reader(lines, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True):
        if len(l) != 9:
            print("Error: ", l)

In [14]:


df = pd.read_csv(csv_file)

# df.head()

518,156,89,7,"Jeep Wrangler JK","This drawing was traced. To trace a drawing, simple go into the settings of the drawing application and add the image URL of the image you would like to trace.",1452,81,https://art.pixilart.com/2f35f5702c.png

1486,91,53,12,"Explore, Conquer","",4,1,https://art.pixilart.com/956a09c2e6.png

2240,256,144,4,"Bromine, Barium","",5,1,https://art.pixilart.com/75094e0c28.png

7131,256,144,4,"give gifts, give life,","",2,3,https://art.pixilart.com/94a6f1c40c.png

7173,256,144,4,"the puppet, #1 in fnafart","",2,1,https://art.pixilart.com/b60dd3f7ac.png

7349,256,144,4,"give gifts, give life,","",1,5,https://art.pixilart.com/be2cdd00be.png

8030,56,33,20,"Alice, Dirt in the Hole.","",2,1,https://art.pixilart.com/35a7e18502.png

12333,256,144,4,"Bryan, Pixilmaster","",1,4,https://art.pixilart.com/b21b6d6d14.png

12902,256,144,4,"Ju-87D Stuka","I apologise for the swastika,but it's a necessary detail",4,2,https://art.pixilart.com/8bbb11fe15.png

13300,256,144,4,"Up

ParserError: Error tokenizing data. C error: Expected 9 fields in line 6694, saw 10
